In [1]:
import sys
import numpy as np
from time import time
from pynq import Overlay
from pynq import allocate
from scipy.linalg import dft

In [2]:
ol=Overlay('DFT.bit')
SIZE = 1024

a = [i for i in range(SIZE)]
a=np.sin(a)
golden_op=np.fft.fft(a)
print(golden_op.real)
print(golden_op.imag)

[0.09084121 0.09084169 0.09084311 ... 0.09084548 0.09084311 0.09084169]
[ 0.          0.00105807  0.00211636 ... -0.00317508 -0.00211636
 -0.00105807]


In [22]:
#内存分配
in_real = allocate(shape=(SIZE,), dtype=np.float32) 
in_img = allocate(shape=(SIZE,), dtype=np.float32)           
out_real = allocate(shape=(SIZE,), dtype=np.float32) 
out_img = allocate(shape=(SIZE,), dtype=np.float32)
np.copyto(in_real, a.real)
np.copyto(in_img, a.imag)

DFT = ol.dft_0
start_time = time()

DFT.write(0x10, out_real.device_address)
DFT.write(0x18, out_img.device_address)
DFT.write(0x20, in_real.device_address)
DFT.write(0x28, in_img.device_address)
DFT.write(0x00, 0x01)

while (DFT.read(0x00) & 0x4) == 0x0:
    continue
end_time = time()
print("耗时：: " + str(end_time - start_time) + " s")
print(out_real)
print(out_img)

耗时：: 0.17218661308288574 s
[0.09084147 0.09084266 0.09084213 ... 0.0908435  0.09084213 0.09084266]
[ 0.          0.00103074  0.00211746 ... -0.00314563 -0.00211746
 -0.00103074]


In [39]:
#计算RMS误差
for i in range(SIZE):
    real_error[i]="{}".format(abs(out_real[i]-golden_op.real[i]))
    imag_error[i]="{}".format(abs(out_img[i]-golden_op.imag[i]))
sum_sq_real=0
sum_sq_imag=0
for i in range(SIZE):
    sum_sq_real +=(real_error[i]*real_error[i])
    real_rmse = np.sqrt(sum_sq_real / (i+1))
    sum_sq_imag +=(imag_error[i]*imag_error[i])
    imag_rmse = np.sqrt(sum_sq_imag / (i+1))
print("Real RMSE: ", real_rmse, "Imag RMSE:", imag_rmse)    
if real_rmse < 0.001 and imag_rmse < 0.001:
    print("PASS")
else:
    print("FAIL")

Real RMSE:  4.874109338230976e-06 Imag RMSE: 1.0921184788292064e-05
PASS


In [36]:
golden_op

array([0.09084121+0.j        , 0.09084169+0.00105807j,
       0.09084311+0.00211636j, ..., 0.09084548-0.00317508j,
       0.09084311-0.00211636j, 0.09084169-0.00105807j])

IndexError: tuple index out of range